In [1]:
import tensorflow as tf
import re
import numpy as np
import os
import time

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
path_to_file = '/content/gdrive/MyDrive/app/procText/lesson_7/evgenyi_onegin.txt'

In [4]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 286984 characters


In [5]:
print(text[:500])

Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                        Вниманье дружбы возлюбя,
                        Хотел бы я тебе представить
                        Залог достойнее тебя,
                        Достойнее души прекрасной,
                        Святой исполненной мечты,
                        Поэзии живой и ясной,
                        Высо


In [ ]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

131 unique characters


In [ ]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [ ]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

А
л
е
к
с


In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '
'      Роман в стихах\n\n                        Не мысля гордый свет забавить,\n                        '
'Вниманье дружбы возлюбя,\n                        Хотел бы я тебе представить\n                        '
'Залог достойнее тебя,\n                        Достойнее души прекрасной,\n                        Свят'
'ой исполненной мечты,\n                        Поэзии живой и ясной,\n                        Высоких д'


In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                         '
Target data: 'лександр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '


In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 300

# Number of RNN units
rnn_units = 512

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [ ]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 131) # (batch_size, sequence_length, vocab_size)


In [ ]:
# sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
# sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [ ]:
# print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
# print()
# print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

In [ ]:
# Train
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 131)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.876278


In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    period=20,
    save_weights_only=True)

In [ ]:
EPOCHS = 200

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/200
44/44 [==============================] - 6s 78ms/step - loss: 2.8204
Epoch 2/200
44/44 [==============================] - 4s 77ms/step - loss: 1.7576
Epoch 3/200
44/44 [==============================] - 4s 77ms/step - loss: 1.5494
Epoch 4/200
44/44 [==============================] - 4s 78ms/step - loss: 1.4055
Epoch 5/200
44/44 [==============================] - 4s 79ms/step - loss: 1.3399
Epoch 6/200
44/44 [==============================] - 4s 79ms/step - loss: 1.2991
Epoch 7/200
44/44 [==============================] - 4s 80ms/step - loss: 1.2589
Epoch 8/200
44/44 [==============================] - 4s 80ms/step - loss: 1.2131
Epoch 9/200
44/44 [==============================] - 4s 80ms/step - loss: 1.1835
Epoch 10/200
44/44 [==============================] - 4s 81ms/step - loss: 1.1487
Epoch 11/200
44/44 [==============================] - 4s 81ms/step - loss: 1.1074
Epoch 12/200
44/44 [==============================] - 4s 82ms/step - loss: 1.0696
Epoch 13/200
44/44 [=====

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_200'

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [ ]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.1

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model, start_string=u"копать "))

копать меня... -
                        "Кому же, милая моя?
                        Как я она!
                        Как величавая луна,
                        Средь жен и дев будто громом поражен.
                        В какую бурю ощущений
                        Теперь яснее - слава богу,
                        По цельным окнам тени ходят,
                        Мелькают профили голов
                        Слова и взор волшебниц сих
                        Обманы, сплетни, кольцы, слезы,



In [250]:
def proc_text_func(i_text):
  # o_proc_text = re.sub(r'[^\w\s]', '', i_text)
  # o_proc_text = re.sub('\n', '', o_proc_text)
  o_proc_text = i_text
  return o_proc_text

In [251]:
proc_text = proc_text_func(text)
words = proc_text.split(' ')
vocab = sorted(set(words))
vocab.remove('')


# text_copy = text
# for word in vocab:
#     text_copy = re.sub(r'\b' + word + r'\b', '', text_copy)
# for char in sorted(set(text_copy))[:13]:
#     vocab.append(char)
# vocab.remove(' ')

In [76]:
# vocab

In [188]:
# text_copy_2 = text
# for word in sorted(set(text_copy))[:13]:
#   if word != ' ':
#     text_copy_2 = re.sub(f'\{word}', ' ' + word + ' ', text_copy_2)
# proc_text = text_copy_2

In [252]:
# Creating a mapping from unique characters to indices
# Присваиваем каждой букве индекс и представляем текст в числовом виде
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

# txt_list = text_copy_2.split('*')
# old_word = ''
# text_as_int = []
# for word in txt_list:
#   if (word in char2idx.keys() and old_word != word):
#     old_word = word
#     text_as_int.append(char2idx[word])
# text_as_int = np.array(text_as_int)

txt_list = proc_text.split(' ')
text_as_int = np.array([char2idx[word] for word in txt_list if word in char2idx.keys()])

In [79]:
# text_as_int

In [253]:
# The maximum length sentence you want for a single input in characters
seq_length = 20
examples_per_epoch = len(proc_text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(20):
    print(idx2char[i.numpy()])

Александр
Сергеевич
Пушкин


Евгений
Онегин

Роман
в
стихах


Не
мысля
гордый
свет
забавить,

Вниманье
дружбы
возлюбя,

Хотел
бы
я
тебе


In [254]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(' '.join(idx2char[item.numpy()])))

'Александр Сергеевич Пушкин\n\n Евгений Онегин\n Роман в стихах\n\n Не мысля гордый свет забавить,\n Вниманье дружбы возлюбя,\n Хотел бы я тебе представить\n'
'Залог достойнее тебя,\n Достойнее души прекрасной,\n Святой исполненной мечты,\n Поэзии живой и ясной,\n Высоких дум и простоты;\n Но так и быть'
'- рукой пристрастной\n Прими собранье пестрых глав,\n Полусмешных, полупечальных,\n Простонародных, идеальных,\n Небрежный плод моих забав,\n Бессонниц, легких вдохновений,\n Незрелых и увядших'
'лет,\n Ума холодных наблюдений\n И сердца горестных замет.\n\n ГЛАВА ПЕРВАЯ\n\n И жить торопится и чувствовать спешит.\n Кн. Вяземский.\n\n I\n\n "Мой дядя'
'самых честных правил,\n Когда не в шутку занемог,\n Он уважать себя заставил\n И лучше выдумать не мог.\n Его пример другим наука;\n'


In [255]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [256]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(' '.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(' '.join(idx2char[target_example.numpy()])))

Input data:  'Александр Сергеевич Пушкин\n\n Евгений Онегин\n Роман в стихах\n\n Не мысля гордый свет забавить,\n Вниманье дружбы возлюбя,\n Хотел бы я тебе'
Target data: 'Сергеевич Пушкин\n\n Евгений Онегин\n Роман в стихах\n\n Не мысля гордый свет забавить,\n Вниманье дружбы возлюбя,\n Хотел бы я тебе представить\n'


In [257]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 20), (64, 20)), types: (tf.int64, tf.int64)>

In [258]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 300

# Number of RNN units
rnn_units = 2048

In [259]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        # tf.keras.layers.GRU(rnn_units,
        #                     return_sequences=True,
        #                     stateful=True,
        #                     recurrent_initializer='glorot_uniform'),

         tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [260]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [261]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 20, 12156) # (batch_size, sequence_length, vocab_size)


In [262]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [263]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 20, 12156)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       9.405495


In [264]:
model.compile(optimizer='adam', loss=loss)

In [265]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    period=20,
    save_weights_only=True)

In [266]:
EPOCHS = 50

In [267]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/50
17/17 [==============================] - 5s 201ms/step - loss: 9.3677
Epoch 2/50
17/17 [==============================] - 3s 204ms/step - loss: 8.7808
Epoch 3/50
17/17 [==============================] - 4s 205ms/step - loss: 8.3901
Epoch 4/50
17/17 [==============================] - 4s 208ms/step - loss: 8.0668
Epoch 5/50
17/17 [==============================] - 4s 210ms/step - loss: 7.8035
Epoch 6/50
17/17 [==============================] - 4s 211ms/step - loss: 7.4222
Epoch 7/50
17/17 [==============================] - 4s 212ms/step - loss: 6.9371
Epoch 8/50
17/17 [==============================] - 4s 211ms/step - loss: 6.4800
Epoch 9/50
17/17 [==============================] - 4s 210ms/step - loss: 5.6892
Epoch 10/50
17/17 [==============================] - 4s 208ms/step - loss: 4.9362
Epoch 11/50
17/17 [==============================] - 4s 207ms/step - loss: 3.9788
Epoch 12/50
17/17 [==============================] - 4s 205ms/step - loss: 3.1712
Epoch 13/50
17/17 [======

In [268]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_40'

In [269]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [270]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 200

    # Converting our start string to numbers (vectorizing)
    # input_eval = [char2idx[s] for s in start_string]

    # text_copy_2 = start_string
    # for word in sorted(set(text_copy))[:13]:
    #     text_copy_2 = re.sub(f'\{word}', '*' + word + '*', text_copy_2)
    prc_txt = proc_text_func(start_string)
    input_eval = [char2idx[s] for s in prc_txt.split(' ') if s in char2idx.keys()]
    # txt_list = text_copy_2.split(' ')
    # input_eval = np.array([char2idx[word] for word in txt_list if word in char2idx.keys()])
    # if start_string in char2idx.keys():
    #   input_eval = [char2idx[start_string]]
    # else:
    #   input_eval = [0]
    input_eval = tf.expand_dims(input_eval, 0)
    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.1

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)
        # print(predicted_id)
        text_generated.append(idx2char[predicted_id])

    return (start_string + ' '.join(text_generated))

In [226]:
# Данный эксперимент проводился с включением в словарь знаков припянания как отдельных слов
print(generate_text(model, start_string=u"Достойнее души прекрасной покой и сон "))

Достойнее души прекрасной покой и сон                                                                                                                                                                                                        


In [249]:
# Данный эксперимент проводился без наков припянания в словаре
print(generate_text(model, start_string=u"Достойнее души прекрасной покой и сон "))

Достойнее души прекрасной покой и сон И бьется радужным крылом Плененный школьным шалуном Так зайчик в озими трепещет Увидя вдруг издалека В кусты пишет страстное посланье Хоть толку мало вообще Он в письмах видел не вотще Но знать сердечное страданье Уже пришло ему в душистых сединах Старик постарому шутивший Отменно тонко и умно Что нынче несколько смешно XXV Тут был на эпиграммы падкий На чердаке вралем рожденной И светской чернью ободренной Что нет нелепицы такой Ни эпиграммы площадной Которой бы ваш друг с девой молодой Оно как будто разрывалось Она глядит ему в лицо Что с вами Так И на крыльцо XX Домой долго он у Москве В долгих иль на почтовых Тянитесь из застав градских V И вы читатель благосклонный В своей коляске выписной Оставьте град неугомонный На ту чей вид напомнил смутно Ему забытые черты Скажи мне князь не знаешь ты Кто там в малиновом погружены XIV Татьяна в лес медведь за нею Снег рыхлый по колено ей То длинный сук ее за шею Зацепит лая кинулись собаки На крик испуга

In [271]:
# Слова и знаки припянания не отдилялись друг от друга
print(generate_text(model, start_string=u"Достойнее души прекрасной покой и сон "))

Достойнее души прекрасной покой и сон бежит;
 Здоровье, жизни цвет и сладость,
 Улыбка, девственный покой,
 Пропало все, что звук пустой,
 И меркнет милой Тани младость:
 Так уж готова
 Бедняжка в обморок упасть;
 Но воля и рассудка власть
 Превозмогли. Она два слова
 Сквозь зубы молвила тишком
 И долго сердцу грустно было.
 "Рооr Yorick! {16} - молвил он уныло. -
 Он на руках меня держал.
 Как часто ласковая муза
 Мне услаждала путь немой
 Волшебством тайного рассказа!
 Как часто по скалам Кавказа
 Она Ленорой, при письмо Все судьбой?

 XVI

 Мечтам и годам нет возврата;
 Не обновлю души моей...
 Я вас люблю любовью брата
 И, может быть, его очей
 Был чудно нежен. Оттого ли,
 Что он и вправду тронут был,
 Иль он, кокетствуя, шалил,
 Невольно ль, иль изменился,
 Расстался б с музами, женился,
 В деревне, счастлив и рогат,
 Носил бы стеганый халат;
 Узнал бы жизнь на самом ядовитый
 Точил лилеи стебелек;
 Чтобы двухутренний цветок
 Увял еще полураскрытый".
 Все это значило, друзья:
 С п

In [ ]:
# Обе модели работают достаточно хорошо. Качество модели зависит от выбора параметров и насколько хорошо составлен словарь.

Поэкспрементировать с переводом текстов к сожалению пока не получается.